In [3]:
import os
import logging
from openai import AzureOpenAI
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from dotenv import load_dotenv
from opentelemetry.trace.status import Status, StatusCode
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient

load_dotenv()
logging.getLogger("azure").setLevel(logging.DEBUG)

PROJECT_CONNECTION_STRING = os.environ["PROJECT_CONNECTION_STRING"]
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

# Setup tracing
tracer = trace.get_tracer("openai_function_tracer")

project_client = AIProjectClient.from_connection_string(
    credential=AzureCliCredential(),
    conn_str=PROJECT_CONNECTION_STRING
)
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)
project_client.telemetry.enable()

def track_exception(span, exc):
    span.set_status(Status(StatusCode.ERROR, str(exc)))
    span.record_exception(exc)

# ---- Function tool setup ----
tool_def = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the weather for a location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string"},
        },
        "required": ["location"],
    },
}

# ---- Execution flow with tracing ----
with tracer.start_as_current_span("openai_function_call_chain") as span:
    try:
        with tracer.start_as_current_span("client_initialization") as s:
            client = AzureOpenAI(
                api_key=AZURE_OPENAI_API_KEY,
                api_version="2025-03-01-preview",
                azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
            )
            s.set_attribute("endpoint", os.getenv("AZURE_OPENAI_ENDPOINT"))

        with tracer.start_as_current_span("initial_response") as s:
            s.set_attribute("model", "gpt-4o")
            s.set_attribute("user_input", "What's the weather in San Francisco?")
            response = client.responses.create(
                model="gpt-4o",
                tools=[tool_def],
                input=[{"role": "user", "content": "What's the weather in San Francisco?"}],
            )
            s.set_attribute("response.id", response.id)
            print(response.model_dump_json(indent=2))

        input = []
        for output in response.output:
            if output.type == "function_call":
                with tracer.start_as_current_span("function_call_execution") as s:
                    s.set_attribute("function_name", output.name)
                    s.set_attribute("call_id", output.call_id)
                    if output.name == "get_weather":
                        function_result = '{"temperature": "70 degrees"}'
                        input.append({
                            "type": "function_call_output",
                            "call_id": output.call_id,
                            "output": function_result
                        })
                        s.set_attribute("function_output", function_result)
                    else:
                        err_msg = f"Unknown function call: {output.name}"
                        s.set_status(Status(StatusCode.ERROR, err_msg))
                        raise ValueError(err_msg)

        with tracer.start_as_current_span("second_response") as s:
            second_response = client.responses.create(
                model="gpt-4o",
                previous_response_id=response.id,
                input=input
            )
            s.set_attribute("response.id", second_response.id)
            print(second_response.model_dump_json(indent=2))

    except Exception as e:
        track_exception(span, e)
        raise


{
  "id": "resp_67f395bf6cb081908f3be0a0f03e624c",
  "created_at": 1744016831.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o",
  "object": "response",
  "output": [
    {
      "arguments": "{\"location\":\"San Francisco\"}",
      "call_id": "call_7xsyefoO649Eh51skkTI8Azo",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_67f395c061048190bf3e480ad858352a",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string"
          }
        },
        "required": [
          "location"
        ]
      },
      "strict": true,
      "type": "function",
      "description": "Get the weather for a location"
    }
  ],
  "top_p": 1.0,
  "max_output_tokens": null,
  "previous_resp